In [1]:
import torch

dbarf = torch.load('/home/gyy/Downloads/dgaussin_train/dgaussian/checkpoint/dbarf_model_200000.pth',map_location=torch.device('cuda:0'))
pixelsplat = torch.load('/home/gyy/Downloads/3500.ckpt',map_location=torch.device('cuda:0'))
print(dbarf.keys())
new_model = {}
new_model['pose_learner'] = dbarf['pose_learner']
new_model['gaussian'] = pixelsplat['state_dict']
torch.save(new_model,'./model_zoo/dgaussian_resnet.pth')

/home/gyy/anaconda3/envs/splat/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


dict_keys(['optimizer', 'scheduler', 'pose_optimizer', 'pose_scheduler', 'net_coarse', 'feature_net', 'pose_learner'])


In [3]:
# new_model['gaussian'].keys()
import fnmatch
a=fnmatch.filter(new_model['gaussian'].keys(),'encoder.backbone.local_token_mlp.*')
b=fnmatch.filter(new_model['gaussian'].keys(),'encoder.backbone.dino.*')
c=fnmatch.filter(new_model['gaussian'].keys(),'encoder.backbone.global_token_mlp.*')
# print(a) encoder.backbone.dino. encoder.backbone.global_token_mlp
for key in a:
    new_model['gaussian'].pop(key)
for key in b:
    new_model['gaussian'].pop(key)
for key in c:
    new_model['gaussian'].pop(key)
new_model['gaussian'].keys()
torch.save(new_model,'./model_zoo/dgaussian_resnet.pth')
# del new_model['gaussian']['encoder.backbone.dino.*']
# new_model['gaussian'].keys()

In [3]:
import torch

origin = torch.load('./model_zoo/dgaussian_resnet.pth',map_location=torch.device('cuda:0'))
dbarf0 = torch.load('./data/ibrnet/train/out/depth_room/model/model_000000.pth',map_location=torch.device('cuda:0'))
dbarf500 = torch.load('./data/ibrnet/train/out/depth_room/model/model_000500.pth',map_location=torch.device('cuda:0'))
dbarf2000 = torch.load('data/ibrnet/train/out/no_depth_room_lr/model/model_002000.pth',map_location=torch.device('cuda:0'))


In [5]:
dbarf500.keys()

dict_keys(['step', 'pose_learner', 'gaussian', 'pose_optimizer', 'pose_scheduler'])

In [ ]:

attn = attn > 0

In [ ]:
import matplotlib.pyplot as plt
fig,a =  plt.subplots(h,w)
for i, per_img_masks in enumerate(attn):
    for j, per_mask in enumerate(per_img_masks):
        norm_mask = per_mask / np.linalg.norm(per_mask)
        a[i][j].set_axis_off()
        a[i][j].imshow(norm_mask)
plt.axis("off")
plt.savefig('out/feat_attn.pdf', bbox_inches='tight')

In [ ]:
import torch
import numpy as np
feat_2d = torch.from_numpy(np.load("out/feat_2d.npy")).to("cuda:1")
roi = feat_2d[250:252, 190:192, 32:]

keys = out / out.norm(dim=0, keepdim=True)
queries = roi / roi.norm(dim=1, keepdim=True)
attn = queries @ keys.reshape(keys.shape[0], -1)
h, w = attn.shape[:2]
attn = attn.reshape(h, w, 192, 252).to('cpu').detach().numpy()


In [ ]:
import torch

# 定义 query 和 key tensor
query = torch.randn(1, 13, 17, 256)
key = torch.randn(1, 100, 200, 256)

# 将 query 和 key 分别转换为形状为 (1, 256, 13, 17) 和 (1, 256, 100, 200) 的 tensor
query = query.permute(0, 3, 1, 2)
key = key.permute(0, 3, 1, 2)

# 使用 2D 卷积计算 correlation map
corr_map = torch.nn.functional.conv2d(key, query, stride=1)

# 转换 correlation map 的形状为 (1, 13, 17, 100, 200)
corr_map = corr_map.permute(0, 2, 3, 1)


In [ ]:
norm_mask = attn[6][6] # / np.linalg.norm(attn[1][5])
np.min(norm_mask)
# norm_mask = norm_mask > 0.9
plt.imshow(norm_mask)
plt.colorbar()

In [ ]:
keys.unique()

In [ ]:
np.unique(attn)

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
for pose in all_poses:
    # 姿态的位置信息
    position = pose[:3, 3]
    x, y, z = position

    # 姿态的旋转信息
    rotation = pose[:3, :3]

    # 绘制位置
    ax.scatter(x, y, z, c='r', marker='o')

    # 绘制方向箭头
    arrow_length = 0.1
    ax.quiver(x, y, z, rotation[0, 0], rotation[1, 0], rotation[2, 0], length=arrow_length, color='r')
    ax.quiver(x, y, z, rotation[0, 1], rotation[1, 1], rotation[2, 1], length=arrow_length, color='g')
    ax.quiver(x, y, z, rotation[0, 2], rotation[1, 2], rotation[2, 2], length=arrow_length, color='b')


# # 设置坐标轴范围
# ax.set_xlim([xmin, xmax])
# ax.set_ylim([ymin, ymax])
# ax.set_zlim([zmin, zmax])

# 显示图形
plt.show()



In [ ]:
import numpy as np
np.save("outputs/depth_room.npy", depth.cpu().detach().numpy())

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
predicted_labels = np.load("out/predicted_labels.npy")
true_labels = np.load("out/true_labels.npy")

plt.imshow(true_labels)
plt.show()
plt.imshow(predicted_labels)
plt.show()


In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt
import torch
def calc_pca(emb, vis=False):
    X = emb.flatten(0, -2).cpu().numpy()
    np.random.seed(80)
    pca = PCA(n_components=3)
    pca.fit(X)
    X_rgb = pca.transform(X).reshape(*emb.shape[:2], 3)
    if vis:
        plt.imshow(X_rgb)
        plt.axis("off")
        plt.show()
    return X_rgb

agg_feat_2d = np.load("out/agg_feat_2d.npy")
agg_feat_2d = torch.from_numpy(agg_feat_2d)
calc_pca(agg_feat_2d, vis=True)

cnn_feat_2d = np.load("out/cnn_feat_2d.npy")
cnn_feat_2d = torch.from_numpy(cnn_feat_2d).permute(1,2,0)
calc_pca(cnn_feat_2d, vis=True)

In [ ]:
agg_feat_2d = np.load("out/sems.npy")[0]
agg_feat_2d = torch.from_numpy(agg_feat_2d)
calc_pca(agg_feat_2d, vis=True)

cnn_feat_2d = np.load("/home/hao/lihao/GNT/out/sems.npy")[0]
cnn_feat_2d = torch.from_numpy(cnn_feat_2d)
calc_pca(cnn_feat_2d, vis=True)

import torch.nn.functional as F
print(F.cosine_similarity(cnn_feat_2d, agg_feat_2d))